# Notebook sniff from Chap 7 Exploring and Cleaning the Bulldozer Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from rfpimp import *  # feature importance plot

In [ ]:
bookcolors = { # our usual book color palette
         'crimson': '#a50026', 'red': '#d73027',
         'redorange': '#f46d43', 'orange': '#fdae61',
         'yellow': '#fee090', 'sky': '#e0f3f8',
         'babyblue': '#abd9e9', 'lightblue': '#74add1',
         'blue': '#4575b4', 'purple': '#313695'}

In [ ]:
df_raw = pd.read_feather("data/bulldozer-train.feather")
df = df_raw.copy()

In [ ]:
def sniff(df):
    with pd.option_context("display.max_colwidth", 20):
        info = pd.DataFrame()
        info['sample'] = df.iloc[0]
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        return info.sort_values('data type')

In [ ]:
basefeatures = ['SalesID', 'MachineID', 'ModelID',
                'datasource', 'YearMade',
                # some missing values but use anyway:
                'auctioneerID', 'MachineHoursCurrentMeter']

In [ ]:
def test(X, y, n_estimators=50):
    rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1, oob_score=True)
    rf.fit(X, y)
    oob = rf.oob_score_
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 {oob:.5f} using {n:,d} tree nodes with {h} median tree height")
    return rf, oob

In [ ]:
X, y = df[basefeatures], df['SalePrice']
X = X.fillna(0) # flip missing numeric values to zeros
rf, oob_baseline_initial = test(X, y)

In [ ]:
df = df.iloc[-100_000:] # take only last 100,000 records

In [ ]:
X, y = df[basefeatures], df['SalePrice']
X = X.fillna(0)
rf, oob_baseline = test(X, y)

In [ ]:
I = importances(rf, X, y)
plot_importances(I)

In [ ]:
del df['MachineID'] # dataset has inconsistencies
del df['SalesID']   # unique sales ID so not generalizer

In [ ]:
df['auctioneerID'] = df['auctioneerID'].astype(str)

In [ ]:
from pandas.api.types import is_string_dtype, is_object_dtype
def df_normalize_strings(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna(np.nan) # make None -> np.nan
            df[col] = df[col].replace('none or unspecified', np.nan)
            df[col] = df[col].replace('none', np.nan)
            df[col] = df[col].replace('#name?', np.nan)
            df[col] = df[col].replace('', np.nan)

In [ ]:
df_normalize_strings(df)
print(df['Drive_System'].unique())
print(df['Backhoe_Mounting'].unique())

In [ ]:
print(df['Tire_Size'].unique())
print(df['Undercarriage_Pad_Width'].unique())

In [ ]:
def extract_sizes(df, colname):
    df[colname] = df[colname].str.extract(r'([0-9.]*)', expand=True)
    df[colname] = df[colname].replace('', np.nan)
    df[colname] = pd.to_numeric(df[colname])

In [ ]:
extract_sizes(df, 'Tire_Size')
extract_sizes(df, 'Undercarriage_Pad_Width')
print(df['Tire_Size'].unique())
print(df['Undercarriage_Pad_Width'].unique())

In [ ]:
print(df['Blade_Width'].unique())
print(df['Stick_Length'].unique())

In [ ]:
from pandas.api.types import is_categorical_dtype
def df_string_to_cat(df):
    for col in df.columns:
        if is_string_dtype(df[col]):
            df[col] = df[col].astype('category').cat.as_ordered()

def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1

In [ ]:
df_toy = pd.DataFrame(data={'Name':['Xue',np.nan,'Tom']})

In [ ]:
df_string_to_cat(df_toy)
df_toy['catcodes'] = df_toy['Name'].cat.codes

In [ ]:
df_cat_to_catcode(df_toy)

In [ ]:
df_string_to_cat(df)
df_cat_to_catcode(df)

In [ ]:
def fix_missing_num(df, colname):
    df[colname+'_na'] = pd.isnull(df[colname])
    df[colname].fillna(df[colname].median(), inplace=True)

In [ ]:
df_toy = pd.DataFrame(data={'YearMade':[1995,2001,np.nan]})

In [ ]:
fix_missing_num(df_toy, 'YearMade')

In [ ]:
print(f"Values {df['Tire_Size'].unique()}")
print(f"Median {df['Tire_Size'].median()}")

In [ ]:
fix_missing_num(df, 'Tire_Size')

In [ ]:
fix_missing_num(df, 'Undercarriage_Pad_Width')

In [ ]:
df_small = df.sample(n=5_000) # don't draw too many dots
df_small.plot.scatter('YearMade','SalePrice', alpha=0.02, c=bookcolors['blue'])

In [ ]:
# There are some unlikely 1919, 1920 values too
# Assume < 1950 is "unknown"
df.loc[df.YearMade<1950, 'YearMade'] = np.nan
fix_missing_num(df, 'YearMade')

In [ ]:
df_small = df.sample(n=5_000)
df_small.plot.scatter('YearMade','SalePrice', alpha=0.02, c=bookcolors['blue'])

In [ ]:
inverted = df.query("saledate.dt.year < YearMade")[['SalePrice','YearMade','saledate']]

In [ ]:
df.loc[df.eval("saledate.dt.year < YearMade"), 'YearMade'] = df['saledate'].dt.year

In [ ]:
df.query("MachineHoursCurrentMeter==0")['YearMade'].plot.hist(bins=30)

In [ ]:
df.loc[df.eval("MachineHoursCurrentMeter==0"),
       'MachineHoursCurrentMeter'] = np.nan
fix_missing_num(df, 'MachineHoursCurrentMeter')

In [ ]:
X, y = df.drop(['SalePrice','saledate'], axis=1), df['SalePrice']
rf, oob_all = test(X, y)

In [ ]:
df = df.reset_index(drop=True)
df.to_feather("data/bulldozer-train-clean.feather")

In [ ]:
I = importances(rf, X, y)
plot_importances(I)